# Principio Abierto/Cerrado (Open/Closed Principle)

## Introducción
El principio abierto/cerrado (OCP) establece que las entidades de software (clases, módulos, funciones) deben estar abiertas para su extensión, pero cerradas para su modificación. Esto permite agregar nuevas funcionalidades sin alterar el código existente.

## Objetivos
- Comprender el principio abierto/cerrado y su impacto en la evolución del software.
- Identificar ejemplos de violaciones y buenas implementaciones del OCP en Python.
- Aplicar el OCP para facilitar la escalabilidad y el mantenimiento del código.

## Ejemplo de la vida real
Piensa en un enchufe múltiple: puedes conectar nuevos dispositivos (extensión) sin modificar el enchufe original (código base).

# Principio de Abierto/Cerrado (Open/Closed Principle, OCP)

## Introducción

El Principio de Abierto/Cerrado (OCP) es uno de los cinco principios SOLID de diseño orientado a objetos. Fue introducido por Bertrand Meyer y establece que las entidades de software (clases, módulos, funciones, etc.) deben estar abiertas para la extensión, pero cerradas para la modificación.

## Explicación Detallada

### Definición

- **OCP**: Una entidad de software debe permitir su extensión sin necesidad de modificar su código fuente original.

### Beneficios del OCP

1. **Mantenibilidad**: Facilita la incorporación de nuevas funcionalidades sin alterar el código existente.

2. **Reusabilidad**: Promueve la creación de componentes reutilizables y extensibles.

3. **Estabilidad**: Minimiza el riesgo de introducir errores en el código existente al agregar nuevas funcionalidades.

## Ejemplos Explicados

### Ejemplo Correcto

Supongamos que estamos desarrollando una aplicación para calcular el área de diferentes formas geométricas. Aplicando el OCP, podríamos tener las siguientes clases:

In [5]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self) -> float:
        pass

class Rectangle(Shape):
    def __init__(self, width: float, height: float) -> None:
        self.width: float = width
        self.height: float = height

    def area(self) -> float:
        return self.width * self.height

class Circle(Shape):
    def __init__(self, radius: float) -> None:
        self.radius = radius

    def area(self) -> float:
        return 3.14159 * (self.radius ** 2)

In [ ]:
# Ejemplo de uso
shapes: list[Shape] = [Rectangle(3, 4), Circle(5)]
for shape in shapes:
    print(f"Área: {shape.area()}")

Área: 12
Área: 78.53975


#### Análisis del Ejemplo Correcto

- **Shape**: Es una clase abstracta que define el método `area`.

- **Rectangle** y **Circle**: Son clases concretas que implementan el método `area`.

Este diseño permite agregar nuevas formas geométricas sin modificar las clases existentes.

### Ejemplo de Violación del OCP

Veamos un ejemplo donde se viola el OCP:

In [3]:
class Shape:
    def __init__(self, shape_type: str, width: float = 0, height: float = 0, radius: float = 0) -> None:
        self.shape_type: str = shape_type
        self.width: float = width
        self.height: float = height
        self.radius: float = radius

    def area(self) -> float:
        if self.shape_type == "rectangle":
            return self.width * self.height
        elif self.shape_type == "circle":
            return 3.14159 * (self.radius ** 2)
        else:
            return 0

In [4]:
# Ejemplo de uso
rectangle = Shape(shape_type="rectangle", width=3, height=4)
circle = Shape(shape_type="circle", radius=5)
shapes: list[Shape] = [rectangle, circle]
for shape in shapes:
    print(f"Área: {shape.area()}")

Área: 12
Área: 78.53975




#### Análisis del Ejemplo Incorrecto

- **Shape**: Tiene múltiples responsabilidades y debe ser modificada cada vez que se agrega una nueva forma geométrica.
- **Rectangle** y **Circle**: No se pueden agregar nuevas formas geométricas sin modificar la clase `Shape`.

Este diseño viola el OCP porque cualquier cambio en las formas geométricas requiere modificar la clase `Shape`.


## Conclusión

1. **Extensibilidad**: El OCP facilita la extensión del software sin modificar el código existente.

2. **Reducción de Errores**: Minimiza el riesgo de introducir errores al agregar nuevas funcionalidades.

3. **Mantenibilidad**: Mejora la mantenibilidad del código al permitir la incorporación de nuevas características de manera aislada.

Aplicar el OCP puede requerir una planificación cuidadosa y el uso de patrones de diseño adecuados, pero los beneficios en términos de estabilidad y extensibilidad del software son significativos.

## Ejercicios prácticos y preguntas de reflexión

1. **Extiende sin modificar**: Observa una función o clase que requiera cambios cada vez que se agrega un nuevo caso. ¿Cómo podrías aplicar el OCP para evitar modificar el código existente?
2. **Refactoriza**: Implementa una solución usando herencia o composición para permitir la extensión de funcionalidades.
3. **Pregunta de reflexión**: ¿Qué riesgos existen si modificamos código ya probado y en producción?

## Autoevaluación
- ¿Puedo agregar nuevas funcionalidades sin modificar el código base?
- ¿Qué patrones de diseño ayudan a cumplir el OCP?

## Referencias y recursos
- [Open/Closed Principle – Wikipedia](https://en.wikipedia.org/wiki/Open%E2%80%93closed_principle)
- [SOLID Principles en Python – Real Python](https://realpython.com/solid-principles-python/)
- [Patrones de diseño y OCP – Refactoring Guru](https://refactoring.guru/es/design-patterns/open-closed-principle)

1. EJERCICIO PRÁCTICO 
----------------------------------------------
-*Uso correcto:*

In [7]:
from abc import ABC, abstractmethod

class Figura(ABC):
    @abstractmethod
    def perimetro(self) -> float:
        pass

class Rectangulo(Figura):
    def __init__(self, ancho: float, alto: float) -> None:
        self.ancho = ancho
        self.alto = alto

    def perimetro(self) -> float:
        return 2 * (self.ancho + self.alto)

class Circulo(Figura):
    def __init__(self, radio: float) -> None:
        self.radio = radio

    def perimetro(self) -> float:
        return 2 * 3.14159 * self.radio

class Triangulo(Figura):
    def __init__(self, lado1: float, lado2: float, lado3: float) -> None:
        self.lado1 = lado1
        self.lado2 = lado2
        self.lado3 = lado3

    def perimetro(self) -> float:
        return self.lado1 + self.lado2 + self.lado3


# ----------------------
triangulo = Triangulo(3, 4, 5)
print(f"Perímetro del triángulo: {triangulo.perimetro():.2f}")
rectangulo = Rectangulo(2, 6)
print(f"Perímetro del rectángulo: {rectangulo.perimetro():.2f}")
circulo = Circulo(4)
print(f"Perímetro del círculo: {circulo.perimetro():.2f}")


Perímetro del triángulo: 12.00
Perímetro del rectángulo: 16.00
Perímetro del círculo: 25.13


2. EJEMPLO 2
---------------------------------
*Uso incorrecto:*


In [9]:
class Figura:
    def __init__(self, tipo: str, ancho: float = 0, alto: float = 0,
                 radio: float = 0, lado1: float = 0, lado2: float = 0, lado3: float = 0) -> None:
        self.tipo = tipo
        self.ancho = ancho
        self.alto = alto
        self.radio = radio
        self.lado1 = lado1
        self.lado2 = lado2
        self.lado3 = lado3

    def perimetro(self) -> float:
        if self.tipo == "rectangulo":
            return 2 * (self.ancho + self.alto)
        elif self.tipo == "circulo":
            return 2 * 3.14159 * self.radio
        elif self.tipo == "triangulo":
            return self.lado1 + self.lado2 + self.lado3
        else:
            return 0
# ----------------------
rect = Figura("rectangulo", ancho=2, alto=6)
print(f"Perímetro rectángulo: {rect.perimetro()}")

circ = Figura("circulo", radio=4)
print(f"Perímetro círculo: {circ.perimetro()}")

tri = Figura("triangulo", lado1=3, lado2=4, lado3=5)
print(f"Perímetro triángulo: {tri.perimetro()}")


Perímetro rectángulo: 16
Perímetro círculo: 25.13272
Perímetro triángulo: 12


*En este ejemplo se viola OCP porque no se debe modificar sino extender y si quisieramos agregar otra figura tocaría modificar el método perimetro*

